In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 19 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

27 February 2023 ; 12:58:55


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,27,0,9.7,1.6,3.6,0.449,...,3.65,114.6,111.05,3.55,3.57,114.68,111.19,3.49,1,1
1,Bobby Portis,PF,27,MIL,49,15,26.6,5.9,11.9,0.501,...,3.65,114.6,111.05,3.55,3.57,114.68,111.19,3.49,0,1
5,Jae Crowder,SF,32,MIL,2,0,20.5,2.5,4.0,0.625,...,3.65,114.6,111.05,3.55,3.57,114.68,111.19,3.49,0,1
7,Joe Ingles,SF,35,MIL,27,0,22.6,2.1,5.4,0.381,...,3.65,114.6,111.05,3.55,3.57,114.68,111.19,3.49,0,1
9,Khris Middleton,SF,31,MIL,19,7,21.4,4.8,11.5,0.417,...,3.65,114.6,111.05,3.55,3.57,114.68,111.19,3.49,0,1


In [5]:
# Set training data
train_data = train_data[train_data['Year'] < 2020] # DELETE THIS LINE
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Malcolm Brogdon,25.4,52,0,BOS,18.7,2.680769,7.096154,14.7,1,2.8,4.573077,0.331604
1,Christian Wood,27.4,50,17,DAL,21.1,2.132000,6.560000,17.6,6,2.2,3.772000,0.208723
2,Bobby Portis,26.6,49,15,MIL,18.8,1.840816,7.028571,14.4,2,1.7,3.346939,0.178820
3,Grant Williams,27.5,60,21,BOS,10.6,0.683333,5.056667,8.6,1,-1.5,2.323333,0.089815
4,Malik Monk,22.8,57,0,SAC,16.6,0.863158,3.308772,14.0,3,1.4,2.157895,0.056935


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)